In [2]:
%load_ext autoreload
%autoreload 2



In [3]:
%run experiment.ipynb


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from mlutils.mlflow.utils import create_runs_for_params, get_unfinished_run_ids, get_runs, get_unfinished_runs, get_run_params, experiment_name_to_id
from glob import glob
from mlutils.distributedcomputing.experiments import run_experiments
from sklearn.model_selection import ParameterGrid
from mlflow import MlflowClient
import mlflow
from pathlib import Path



In [10]:
client = MlflowClient(tracking_uri="sqlite:///experiments.db")
mlflow.set_tracking_uri("sqlite:///experiments.db")


In [5]:
all_processed_data_files = glob(f"../datasets/notebooks/processed/*.csv")

all_file_names = [
    file.split('/')[-1].split('train')[0].rstrip('-') for file in all_processed_data_files if 'tra.csv' in file
]
print(all_file_names[:5])

['shuttle-5-5tra.csv', 'pima-5-5tra.csv', 'wisconsin-5-4tra.csv', 'automobile-5-2tra.csv', 'spambase-5-1tra.csv']


In [6]:
train_file_paths = [it for it in all_processed_data_files if 'tra.csv' in it]
dataset_names = {path.split('/')[-1].split('-')[0] for path in train_file_paths}
selected_datasets = []
for name in dataset_names:
    selected_datasets.append([p for p in train_file_paths if name in p][0])

In [7]:
len(train_file_paths)

100

In [8]:
selected_datasets

['../datasets/notebooks/processed/wine-5-2tra.csv',
 '../datasets/notebooks/processed/vowel-5-5tra.csv',
 '../datasets/notebooks/processed/breast-5-2tra.csv',
 '../datasets/notebooks/processed/mushroom-5-3tra.csv',
 '../datasets/notebooks/processed/pima-5-5tra.csv',
 '../datasets/notebooks/processed/chess-5-5tra.csv',
 '../datasets/notebooks/processed/nursery-5-2tra.csv',
 '../datasets/notebooks/processed/heart-5-5tra.csv',
 '../datasets/notebooks/processed/haberman-5-3tra.csv',
 '../datasets/notebooks/processed/car-5-5tra.csv',
 '../datasets/notebooks/processed/balance-5-5tra.csv',
 '../datasets/notebooks/processed/spambase-5-1tra.csv',
 '../datasets/notebooks/processed/mammographic-5-5tra.csv',
 '../datasets/notebooks/processed/banana-5-4tra.csv',
 '../datasets/notebooks/processed/titanic-5-4tra.csv',
 '../datasets/notebooks/processed/shuttle-5-5tra.csv',
 '../datasets/notebooks/processed/yeast-5-5tra.csv',
 '../datasets/notebooks/processed/ecoli-5-3tra.csv',
 '../datasets/notebooks/

In [11]:
params = ParameterGrid({
    'train_path': selected_datasets,
    'base_clf': ["perceptron", "dt", "knn", "nb"],
    'min_samples': [10, 25],
    "complexity_measure": ["f2", "n3", "l3"],
    "min_split_percentage": [0.1, 0.3, 0.4]
})

In [10]:
base_params = ParameterGrid({
    'train_path': train_file_paths,
})

In [ ]:
base_runs = create_runs_for_params(base_params, experiment_name='base', client=client)


In [12]:
experiment_runs = create_runs_for_params(params, experiment_name='v8', client=client)

2023-10-28 17:42:53.158 | INFO     | mlutils.mlflow.utils:create_runs_for_params:32 - Experiment not found, creating new. Id=7
100%|██████████| 1440/1440 [00:16<00:00, 87.36it/s] 


In [5]:
# experiment_runs = get_runs(experiment_name_to_id("v8", client=client))
# base_runs = get_runs(experiment_name_to_id("base", client=client))

In [14]:
experiment_runs

['7aa1a4d8a2964c86bbb335b4bf5c1081',
 'cf035d3d6aa94712809a24cce90e1526',
 '78ec90a7981a41d4b11078d5d026ee07',
 'bf352017881d4b3097860373e83b7d20',
 '01c8586c82d9408b8a9d264d5ddff691',
 'f877a766e9b44db2a313a153c4bab436',
 '765c0f50d327426b97fb83a791619d45',
 'b00fe8f22cec41459027bf9620ad2933',
 'decdcefa3f314c2d862d4bd29d5e3da7',
 '9277acd6efd647d3a43bb1132d1442dd',
 'd843f01dfc3f4497b376f8b0fa4f9911',
 '2541e6a1cc1f471ca34c616092ea5b96',
 '3931182a93114a57ba8c46d87c4ffa93',
 '25ed205070b74ecb96a0df84bea0c258',
 'db272a7c2e0a48a5b676e76855f95f07',
 'b9c55b78308344c4ad37080c683fe84e',
 '2037b22cfa4142d79807ee87e96a84c3',
 'abd2d592f28f4f8db62dd9510f9a9a2d',
 '23b58782a6ec41809acca5a74d39f57c',
 '232549492f6045febaa19c5c96cefc26',
 'af362fbe6a314327869b16c8a9625f41',
 'd71a8060da404a1b8b5c37674e6a2e31',
 'e8d9a455fbe44b8ba9c5fe6df5deae0b',
 '1b0946222d1144d0a9b940be6373e14b',
 'e7eb09a8ac99415ba3c770e7a86e8ee1',
 '698521b9199542d78710048a73cf61e6',
 '7fed7763524f46f388fdc49007d64d6f',
 

In [16]:
experiment_runs.status.value_counts()

AttributeError: 'list' object has no attribute 'status'

In [ ]:
v1_run_ids = v1_runs.run_id

In [13]:
unfinished_runs = get_unfinished_run_ids(experiment_name_to_id('v8', client=client))

In [14]:
len(unfinished_runs)

1215

In [11]:
len(unfinished_runs)

1918

In [15]:
run_experiments(unfinished_runs, 'do_experiment_v1', notebook_path="experiment.ipynb", output_dir_path=Path("/tmp"), run_in_batches=True, backend='papermill', n_jobs=6)

2023-10-29 17:22:42.685 | INFO     | mlutils.distributedcomputing.experiments:run_experiments:11 - {'notebook_path': 'experiment.ipynb', 'output_dir_path': PosixPath('/tmp'), 'run_in_batches': True, 'n_jobs': 6}
2023-10-29 17:22:42.688 | WARNING  | mlutils.distributedcomputing.experiments:run_experiments:21 - Using Papermill backend. Make sure you execute this function not from experiment notebook!
2023-10-29 17:22:42.694 | INFO     | mlutils.distributedcomputing.job_scheduling:create_experiment_notebook:91 - 
    Creating experiment notebook
        notebook_name=experiment.ipynb
        experiment_function=do_experiment_v1
        instance_id_param_name=EXPERIMENT_INSTANCE_ID
        
2023-10-29 17:22:42.721 | INFO     | mlutils.distributedcomputing.experiments:run_experiments:34 - Notebook path=/Users/bgulowaty/studia/projekty/quad-splits/experiment-b6c97163-c99a-41ba-9467-c294e5a7d7be.ipynb
2023-10-29 17:22:42.783 | INFO     | mlutils.distributedcomputing.job_scheduling:run_in_pape

FileNotFoundError: [Errno 2] No such file or directory: '/Users/bgulowaty/studia/projekty/quad-splits/experiment-b6c97163-c99a-41ba-9467-c294e5a7d7be.ipynb'

In [15]:
run_experiments(unfinished_runs[:3], do, backend='plain')

2023-08-23 23:15:53.371 | INFO     | mlutils.distributedcomputing.experiments:run_experiments:11 - {}
2023-08-23 23:15:53.382 | INFO     | __main__:do_experiment_base:5 - {'train_path': '../datasets/notebooks/processed/banana-5-3tra.csv'}
2023-08-23 23:15:53.774 | INFO     | __main__:do_experiment_base:5 - {'train_path': '../datasets/notebooks/processed/vowel-5-2tra.csv'}
2023-08-23 23:15:53.938 | INFO     | __main__:do_experiment_base:5 - {'train_path': '../datasets/notebooks/processed/nursery-5-5tra.csv'}
2023-08-23 23:15:54.350 | INFO     | __main__:do_experiment_base:5 - {'train_path': '../datasets/notebooks/processed/car-5-2tra.csv'}
2023-08-23 23:15:54.453 | INFO     | __main__:do_experiment_base:5 - {'train_path': '../datasets/notebooks/processed/yeast-5-2tra.csv'}
2023-08-23 23:15:54.607 | INFO     | __main__:do_experiment_base:5 - {'train_path': '../datasets/notebooks/processed/wisconsin-5-1tra.csv'}
2023-08-23 23:15:54.683 | INFO     | __main__:do_experiment_base:5 - {'train_

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]